# What if using the RWR on functionally-based adjacencies

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import itertools as it 
import sklearn
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import normalize

import pandas as pd
import pickle
import plotly
import plotly.graph_objs as pgo
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio

import seaborn as sns
import umap

%matplotlib inline

### UMAP parametric resources
+ https://colab.research.google.com/drive/1lpdCy7HkC5TRI9LfUtIHBBW8oRO86Nvi?usp=sharing
+ https://colab.research.google.com/drive/1lpdCy7HkC5TRI9LfUtIHBBW8oRO86Nvi?usp=sharing#scrollTo=BwAID0P-XWJK
+ https://umap-learn.readthedocs.io/en/latest/parametric_umap.html#saving-and-loading-your-model
+ https://timsainburg.com/parametric-umap.html

--- 
# FUNCTIONS for this script

In [ ]:
# -------------------------------------
# the original function by Felix: 
# -------------------------------------

'''
Random Walk Operator with restart probability.
Return Matrix.
''' 
def rnd_walk_matrix2(A, r, a, num_nodes):

    num = 1*num_nodes
    n = num_nodes
    factor = float((1-a)/n) # = 0 if alpha = 1.0 

    E = np.multiply(factor,np.ones([n,n]))              # prepare 2nd scaling term
    A_tele = np.multiply(a,A) + E  #     print(A_tele)
    M = normalize(A_tele, norm='l1', axis=0)                                 # column wise normalized MArkov matrix

    # mixture of Markov chains
    del A_tele
    del E

    U = np.identity(n,dtype=int) 
    H = (1-r)*M
    H1 = np.subtract(U,H)
    del U
    del M
    del H    

    W = r*np.linalg.inv(H1)   

    return W


# -------------------------------------
# rwr modified (as used in this script) 
# -------------------------------------

def rwr_matrix(A, r): 
    n = len(A) # take the whole network into account = all nodes are seed nodes
    factor = 1 
    a = 1 # maximum freedom to jump anytime to any node within the network
    
    E = np.multiply(factor,np.ones([n,n])) # Matrix with all 1 --> to 
    A_tele = np.multiply(a,A) + E  #     print(A_tele)
    M = normalize(A_tele, norm='l1', axis=0)                                 # column wise normalized MArkov matrix

    # mixture of Markov chains
    del A_tele
    del E

    U = np.identity(n,dtype=int) 
    H = (1-r)*M
    H1 = np.subtract(U,H)
    del U
    del M
    del H    

    W = r*np.linalg.inv(H1)   

    return W


# -------------------------------------
# rwr simulated (just for testing, output = path from random walker)
# -------------------------------------

def random_walk_simple(start_v, number_of_steps):

    for step in range(1, number_of_steps):

        start_vertex = start_v # index of vertex to start from
        visited_vertices = {} # Dictionary that associate nodes with the amount of times it was visited   

        path = [start_vertex] # Store and print path  

        counter = 0 # Restart the cycle
        for counter in range(1, number_of_steps): 
            vertex_neighbors = [n for n in G.neighbors(start_vertex)] # get adjacent nodes
            probability = [] # Set probability of going to a neighbour is uniform
            probability = probability + [1./len(vertex_neighbors)] * len(vertex_neighbors)

            start_vertex = np.random.choice(vertex_neighbors, p=probability) # Choose a vertex from the vertex neighborhood to start the next random walk

            if start_vertex in visited_vertices: # Accumulate the amount of times each vertex is visited
                visited_vertices[start_vertex] += 1
            else:
                visited_vertices[start_vertex] = 1

            path.append(start_vertex) # Append to path

        # mostvisited = sorted(visited_vertices, key = visited_vertices.get,reverse = True) # Organize the vertex list in most visited decrescent order
        # print("Path: ", path)
        # print("Most visited nodes: ", mostvisited[:10]) # Separate the top 10 most visited vertex
        
        return path

    
# -------------------------------------
# plotting functions 
# -------------------------------------   

def draw_node_degree(G, scalef):
    #x = 20
    #ring_frac = np.sqrt((x-1.)/x)
    #ring_frac = (x-1.)/x

    l_size = {}
    for node in G.nodes():
        k = nx.degree(G, node)
        R = scalef * (1 + k**1.1) 

        l_size[node] = R
        
    return l_size


def draw_node_degree_3D(G, scalef):
    x = 3
    ring_frac = (x-1.)/x

    deg = dict(G.degree())
    
    d_size = {}
    for i in G.nodes():
        for k,v in deg.items():
            if i == k:
                R = scalef * (1+v**0.9)
                r = ring_frac * R
                d_size[i] = R
    
    return d_size 


def embed_umap_2D(Matrix, n_neighbors, spread, min_dist, metric='cosine'):
    n_components = 2 # for 2D

    U = umap.UMAP(
        n_neighbors = n_neighbors,
        spread = spread,
        min_dist = min_dist,
        n_components = n_components,
        metric = metric)
    embed = U.fit_transform(Matrix)
    
    return embed


def get_posG_2D(l_nodes, embed):
    posG = {}
    cc = 0
    for entz in l_nodes:
        # posG[str(entz)] = (embed[cc,0],embed[cc,1])
        posG[entz] = (embed[cc,0],embed[cc,1])
        cc += 1

    return posG


def get_posG_3D(l_genes, embed):
    posG = {}
    cc = 0
    for entz in l_genes:
        posG[entz] = (embed[cc,0],embed[cc,1],embed[cc,2])
        cc += 1
    
    return posG


def color_nodes_from_dict(G, d_to_be_coloured, color_method):

    # Colouring
    colour_groups = set(d_to_be_coloured.values())
    colour_count = len(colour_groups)
    pal = sns.color_palette('YlOrRd', colour_count)
    palette = pal.as_hex()

    d_colourgroups = {}
    for n in colour_groups:
        d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
        
    d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

    d_val_col = {}
    for idx,val in enumerate(d_colourgroups_sorted):
        for ix,v in enumerate(palette):
            if idx == ix:
                d_val_col[val] = v

    d_node_colour = {}
    for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
        for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
            if x[0] == y[1]:
                d_node_colour[y[0]]=x[1]

    # SORT dict based on G.nodes
    d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])
    l_col = list(d_node_colour_sorted.values())
    colours = l_col

    return colours

def color_clusters_from_dict(G, d_to_be_coloured, color_method):

    # Colouring
    colour_groups = set(d_to_be_coloured.values())
    colour_count = len(colour_groups)
    pal = sns.color_palette('Spectral', colour_count)
    palette = pal.as_hex()

    d_colourgroups = {}
    for n in colour_groups:
        d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
        
    d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

    d_val_col = {}
    for idx,val in enumerate(d_colourgroups_sorted):
        for ix,v in enumerate(palette):
            if idx == ix:
                d_val_col[val] = v

    d_node_colour = {}
    for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
        for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
            if x[0] == y[1]:
                d_node_colour[y[0]]=x[1]

    # SORT dict based on G.nodes
    d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])
    l_col = list(d_node_colour_sorted.values())
    colours = l_col
    
    return colours


def embed_umap_3D(Matrix, n_neighbors, spread, min_dist, metric='cosine'):
    n_components = 3 # for 3D

    U_3d = umap.UMAP(
        n_neighbors = n_neighbors,
        spread = spread,
        min_dist = min_dist,
        n_components = n_components,
        metric = metric)
    embed = U_3d.fit_transform(Matrix)
    
    return embed


def get_posG_3D(l_genes, embed):
    posG = {}
    cc = 0
    for entz in l_genes:
        posG[entz] = (embed[cc,0],embed[cc,1],embed[cc,2])
        cc += 1
    
    return posG


def get_trace_nodes_3D(posG, info_list, color_list, size):

    key_list=list(posG.keys())
    trace = pgo.Scatter3d(x=[posG[key_list[i]][0] for i in range(len(key_list))],
                           y=[posG[key_list[i]][1] for i in range(len(key_list))],
                           z=[posG[key_list[i]][2] for i in range(len(key_list))],
                           mode = 'markers',
                           text = info_list,
                           hoverinfo = 'text',
                           #textposition='middle center',
                           marker = dict(
                color = color_list,
                size = size,
                symbol = 'circle',
                line = dict(width = 1.0,
                        color = color_list)
            ),
        )
    
    return trace


def get_trace_edges_3D(G, posG, color_list, opac = 0.2):
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        x0, y0, z0 = posG[edge[0]]
        x1, y1, z1 = posG[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_z.append(z0)
        edge_z.append(z1)
        edge_z.append(None)

    trace_edges = pgo.Scatter3d(
                        x = edge_x, 
                        y = edge_y, 
                        z = edge_z,
                        mode = 'lines', hoverinfo='none',
                        line = dict(width = 0.5, color = color_list),
                        opacity = opac
                )
    
    return trace_edges


def color_nodes_same(G, list_color_nodes, color):

    d_col = {}
    for node in list_color_nodes:
            d_col[node] = color
      
    return d_col 

### Get human PPI + functional feature matrix

In [ ]:
organism = 'Human'

G = nx.read_edgelist('input/ppi_elist.txt',data=False)
# d_ent_sym, d_sym_ent = genent2sym()

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

df_gene_sym = pd.read_csv('output_csv/DF_gene_symbol_Human.csv', index_col=0)
l_features = list((df_gene_sym.to_dict()).values())

posG_entrez = []
for k in G.nodes():
    posG_entrez.append(k)
    
    
features_MF = pd.read_csv('output_csv/Features_GO_MolFunc_Dataframe_Human.csv', index_col=0)
feat_genes = list(features_MF.index)

features_MF

### Get Feature top list i.e. features with most genes associated 

In [ ]:
# SUM UP ROWS to see max-gene-count associated with feature-value
feat_array = features_MF.to_numpy()

# GET FEATURES TOP LIST
feat_col_added = [sum(x) for x in zip(*feat_array)]

feat_col_names = list(features_MF.columns)
d_feat_col = dict(zip(feat_col_names,feat_col_added))
d_feat_col_sorted = {k: v for k, v in sorted(d_feat_col.items(), key=lambda item: item[1], reverse=True)}

lower_bound = 100 # 0 # 100
upper_bound = 200 # 10000 # 200
num_of_features = upper_bound-lower_bound


d_feat_toplist = {}
for k,v in d_feat_col_sorted.items():
    if v >= lower_bound and v <= upper_bound: 
        d_feat_toplist[k] = v

print('Number of top features based on selection:',len(d_feat_toplist))
#print('Biological Function(s) basis for "artificial" Adjacency Matrix:', d_feat_toplist)

In [ ]:
# FROM TOPLIST check for genes associated with features  
feat_top_genes = []
for feature,genes in features_MF.iteritems():
    for top in d_feat_toplist.keys():
        if top == feature:
            for k,v in genes.items():
                if v == 1:
                    feat_top_genes.append(int(k))             
                    
feat_top_genes_sort = sorted(feat_top_genes)
print('Number of genes associated with feature function:',len(list(set(feat_top_genes_sort))))

### Construct "Feature-based Adjacency Matrix" ("artificial adjacencies") 

In [ ]:
# construct empty matrix to be filled with values based on feature-associations

# original graph row names
graph_row_names = []
for i in list(G.nodes()):
    graph_row_names.append(int(i))

empty_matrix = np.zeros(shape=(len(graph_row_names),len(graph_row_names)))
empty_df = pd.DataFrame(empty_matrix, index = graph_row_names, columns = graph_row_names)
empty_df

In [ ]:
# construct a matrix with all feature-associated genes (values = 1) 

# feature genes row names
feat_row_names = [str(i) for i in feat_top_genes_sort]

feat_graph_overlapping_genes = []
for i in graph_row_names:
    if str(i) in feat_row_names: 
        feat_graph_overlapping_genes.append(int(i))
            
feat_genes_values_matrix = np.ones([len(feat_graph_overlapping_genes),len(feat_graph_overlapping_genes)])
df_feat_genes = pd.DataFrame(feat_genes_values_matrix, index = feat_graph_overlapping_genes, columns = feat_graph_overlapping_genes)
np.fill_diagonal(df_feat_genes.values, 0)
df_feat_genes

In [ ]:
df1 = df_feat_genes
df2 = empty_df

In [ ]:
# COLUMNS : merging df1 and df2 by replacing 0 to 1 if gene in feature function list

for i,v in df2.iteritems():
    for k in df1.index:
        if i == k:
            df2[i] = 1.0

In [ ]:
df2

In [ ]:
# check if specific column values are overwritten i.e. value = 1 

print(df2.loc[66008,4928])
print(df2.loc[66008,1994])

In [ ]:
# ROWS : merging df1 and df2 by replacing 0 to 1 if gene in feature function list

for i in df2.index:
    for k in df1.index:
        if i == k:
            df2.loc[i,] = 1.0

In [ ]:
# check if specific row values are overwritten i.e. value = 1 
df2.iloc[6]

In [ ]:
df2

In [ ]:
# check if diagonal values are 0
print(df2.loc[1994,1994])

np.fill_diagonal(df2.values, 0)

# check again 
print(df2.loc[1994,1994])

In [ ]:
df_artificial_adjacency = df2
df_artificial_adjacency

In [ ]:
df_artificial_adjacency.loc[66008,4928]

In [ ]:
df_artificial_adjacency.loc[4928,66008]

### Adjacency Matrices - structural + functional for comparison

In [ ]:
# Functional features based Aij
A_features = df_artificial_adjacency.to_numpy()

# Structural features based Aij
A_graph = nx.adjacency_matrix(G)
A = A_graph.toarray()

#### Functional feature-based RWR

In [ ]:
%%time 

r = 0.8 # restart probability to 
func_rwr_matrix = rwr_matrix(A_features,r)


df_func = pd.DataFrame(func_rwr_matrix, columns = list(G.nodes()), index=list(G.nodes()))
df_func

#### Structural/classic RWR

In [ ]:
struct_rwr_matrix = rwr_matrix(A,r)

df_struct = pd.DataFrame(struct_rwr_matrix, columns = list(G.nodes()), index=list(G.nodes()))
df_struct

In [ ]:
# INPUT FOR EMBEDDING ≠ Distance Matrix, but instead FEATURE MATRIX

#DM = df_struct
#feature='structural'

DM = df_func
feature='functional'

# set gene list (= G.nodes())
# all having the same order as G.nodes() 
genes = []
for i in DM.index:
    genes.append(str(i))

genes_rest = [] 
for g in G.nodes():
    if g not in genes:
        genes_rest.append(g)

---
 # VISUALIZATION

In [ ]:
# Node, Edge colors

edge_width = 0.1
edge_color = 'lightgrey'

edge_colordark = 'dimgrey'
opacity_edges = 0.5

opacity_nodes = 1.0
node_edge_color = None


# Node sizes 

#size = 10.0
#size3d = 5.0

# if node size reflects degree : 

scalef= 0.25
size = list(draw_node_degree(G, scalef).values())

scalef= 0.05
size3d = list(draw_node_degree_3D(G, scalef).values())

# Non-parametric UMAP 2D

In [ ]:
n_neighbors = 20 # balance between local and global structure in the data
spread = 1.0
min_dist = 0.1 # defines how dense points are stacked together 
metric ='cosine'

In [ ]:
%%time 

umap_2D = embed_umap_2D(DM, n_neighbors, spread, min_dist, metric)

In [ ]:
%%time 

posG_umap = get_posG_2D(genes, umap_2D)
posG_complete_umap = {key:posG_umap[key] for key in G.nodes()}

# normalize coordinates (for DataDivr Layout input)
x_list = []
y_list = []
for k,v in posG_complete_umap.items():
    x_list.append(v[0])
    y_list.append(v[1])
    
xx_norm = sklearn.preprocessing.minmax_scale(x_list, feature_range=(0, 1), axis=0, copy=True)
yy_norm = sklearn.preprocessing.minmax_scale(y_list, feature_range=(0, 1), axis=0, copy=True)

posG_complete_umap_norm = dict(zip(list(G.nodes()), zip(xx_norm,yy_norm)))

### Node Colour Settings to choose: 

+ 1. Spectral Clustering 

In [ ]:
# Spectral Clustering 

n_clus = 50 #len(d_feat_toplist)
color_method = 'spectral'

model = SpectralClustering(n_clusters=n_clus,n_components = 2, affinity='nearest_neighbors')
clusterid = model.fit_predict(df_posG)

d_node_clusterid = dict(zip(list(G.nodes()), clusterid))

# uncomment to set colours
colours = color_clusters_from_dict(G, d_node_clusterid, color_method)

+ 2. Centrality Metrics 

In [ ]:
# Centrality metrics 
color_method = 'clos'

df_centralities = pd.read_csv('output_csv/Features_centralities_Dataframe_'+organism+'.csv', index_col=0)

d_deghubs = dict(zip(G.nodes(),df_centralities['degs']))
d_clos = dict(zip(G.nodes(), df_centralities['clos']))
d_betw = dict(zip(G.nodes(), df_centralities['betw']))
d_eigen = dict(zip(G.nodes(), df_centralities['eigen']))

# uncomment to set colours
colours = color_nodes_from_dict(G, d_clos, color_method)

+ 3. Feature Based

In [ ]:
# Feature based Colouring (all feature associated nodes = coloured , all other = grey)
color_method = 'feat-based'

graph_nodes_subtracted = []
for i in G.nodes():
    if int(i) not in feat_graph_overlapping_genes:
        graph_nodes_subtracted.append(str(i))

feat_graph_overlapping_genes_str = []
for i in feat_graph_overlapping_genes:
    feat_graph_overlapping_genes_str.append(str(i))

    
col_feat_genes = color_nodes_same(G, feat_graph_overlapping_genes_str, 'orange')
col_nonfeat_genes = color_nodes_same(G, graph_nodes_subtracted, 'grey')

colours_unsorted = {**col_feat_genes, **col_nonfeat_genes}
colours_sorted = {key:colours_unsorted[key] for key in G.nodes()}

# uncomment to set colours
colours = list(colours_sorted.values())

In [ ]:
# print('Functional Aij based on following biological Functions:', d_feat_toplist)
plt.figure(figsize=(25,25))
plt.title(feature+' | UMAP | Metric: '+metric, size=16)

nx.draw_networkx_nodes(G, posG_complete_umap_norm, edgecolors = node_edge_color, linewidths = 0.5, node_color=colours, 
                       node_size = size, 
                       alpha = opacity_nodes)
nx.draw_networkx_edges(G, posG_complete_umap_norm, width = edge_width, edge_color = edge_color, alpha = opacity_edges)
plt.box(False)

plt.savefig('output_plots/2DPortraitumap_'+feature+'_'+metric+'_'+color_method+str(num_of_features)+'_'+str(n_neighbors)+'_'+str(spread)+'_'+str(min_dist)+'_'+organism+'.png')

plt.show()

---- 
# UMAP 3D 

In [ ]:
%%time 

umap_3D = embed_umap_3D(DM, n_neighbors, spread, min_dist, metric)

In [ ]:
posG_3Dumap = get_posG_3D(G.nodes(), umap_3D)
posG_3D_complete_umap = {key:posG_3Dumap[key] for key in G.nodes()}

# normalize coordinates (for DataDivr Layout input)
x_list = []
y_list = []
z_list = []
for k,v in posG_3D_complete_umap.items():
    x_list.append(v[0])
    y_list.append(v[1])
    z_list.append(v[2])
    
xx_norm = sklearn.preprocessing.minmax_scale(x_list, feature_range=(0, 1), axis=0, copy=True)
yy_norm = sklearn.preprocessing.minmax_scale(y_list, feature_range=(0, 1), axis=0, copy=True)
zz_norm = sklearn.preprocessing.minmax_scale(z_list, feature_range=(0, 1), axis=0, copy=True)

posG_3D_complete_umap_norm = dict(zip(list(G.nodes()), zip(xx_norm,yy_norm,zz_norm)))

In [ ]:
%%time 
umap_nodes = get_trace_nodes_3D(posG_3D_complete_umap_norm, l_features, colours, size3d)
umap_edges = get_trace_edges_3D(G, posG_3D_complete_umap_norm, edge_colordark, opac=0.05)

umap_data = [umap_edges, umap_nodes]
#umap_data = [umap_nodes]

In [ ]:
print('# Neighbours:', n_neighbors)
print('Spread:',spread)
print('Min. Distance:', min_dist)


fig = pgo.Figure()
for i in umap_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),    
                    dragmode="turntable",
                    #annotations=annotations,
                ))    


py.iplot(fig)
plotly.offline.plot(fig, filename = 'output_plots/3Dportrait_umap_'+feature+'_'+metric+'_'+color_method+str(num_of_features)+'_'+str(n_neighbors)+'_'+str(spread)+'_'+str(min_dist)+'_'+organism+'.html', auto_open=True)